# Explainable AI
this script includes the scaffold needed to test out explainable AI for the melanoma classification project. access to the following data is given
- a ML model
- input shape expected by that model
- a single image and a corresponding prediction

## Import utils script setup

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
#%pip install --user tensorflow==2.8.3
#%pip install --user keras==2.8.0

In [3]:
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)
import numpy; print(numpy.__version__)

2.8.3
2.8.0
1.21.5


## Getting the model

#### Load model into memory

In [5]:
from tensorflow.keras.models import load_model
from efficientnet.tfkeras import EfficientNetB0

# NOTE: a different model which takes non-flattened data is needed. such a model is currently not on the cluster
model_path = "explain_ai_model/dummy_model.h5"
model = load_model(model_path)

#### Investigate expected shape

In [19]:
from math import sqrt

input_shape = model.get_config()["layers"][0]["config"]["batch_input_shape"]
expect_flattened_input = len(input_shape) == 2
img_pixel = int(sqrt(input_shape[1]/3) if expect_flattened_input else input_shape[1])

In [20]:
are_flat = "are" if expect_flattened_input else "are not"
print(f"the model expects images with size {img_pixel}*{img_pixel} which {are_flat} flattened out")

the model expects images with size 224*224 which are not flattened out


## Load images

#### Config on where to find the data

In [21]:
base_path = "data" 
downsampled_data = False
partition = "test"
num_images = 1

In [22]:
img_folder = base_path + "/" + partition + ("_downsampled" if downsampled_data else "")
groundtruth_file = base_path + "/ISIC_2020_2019_" + partition + ("_downsampled" if downsampled_data else "") + ".csv" 

#### Getting image(s)

In [23]:
from utilities import get_img_paths, load_data, split_predictors_target, unflatten_images_df

img_paths = get_img_paths(img_folder, num_images) 
df = load_data(img_paths, groundtruth_file, img_pixel=img_pixel)
X, y = split_predictors_target(df) 

if not expect_flattened_input:
    X = unflatten_images_df(X, img_pixel=img_pixel)

# Hi Renée
at this point, 
- X includes a list of images which may or may not be flat depending on what the model expects. 
- y includes the correct labels
- model includes the loaded ML model which should be used to make the predicitons.

You can adjust the num of images display with num_images and the model used in model_path. the other stuff should (hopefully) be automatically adjusted

## Predict and display images

In [24]:
import matplotlib.pyplot as plt
depth = 3

# predict images
y_pred = model.predict(X)

if expect_flattened_input:
    i = 0
    for img_name, img in X.iterrows():
        # output prediction
        title = f"Image {i+1}"
        prediction = f"Confidence that the image is melanoma: {y_pred[i][0]}"
        i+=1

        # get image in correct format
        img_to_display = img.to_numpy()
        img_to_display = img_to_display.reshape(img_pixel,img_pixel,depth) # unflatten the image

        # display image
        plt.figure()
        plt.imshow(img_to_display)
        plt.title(prediction)
        plt.suptitle(title, fontweight ="bold")
else: 
    

AttributeError: 'numpy.ndarray' object has no attribute 'iterrows'

In [11]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as c_map
from IPython.display import Image, display
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.xception import Xception, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

import lime
from lime import lime_image
from lime import submodular_pick

from skimage.segmentation import mark_boundaries

np.random.seed(42)

print(f" Version of tensorflow used: {tf.__version__}")

 Version of tensorflow used: 2.8.3


In [ ]:
#image_path => image that will be worked on

In [ ]:
#Image transformation of the example
#IMG_SIZE = (299, 299)
#def transform_image(image_path, size):
#    '''
#    Function to transform an image to normalized numpy array
#    '''
#    img = image.load_img(image_path, target_size=size)
#    img = image.img_to_array(img)# Transforming the image to get the shape as [channel, height, width]
#    img = np.expand_dims(img, axis=0) # Adding dimension to convert array into a batch of size (1,299,299,3)
#    img = img/255.0 # normalizing the image to keep within the range of 0.0 to 1.0
    
#    return img

#normalized_img = transform_image(image_path, IMG_SIZE)

In [ ]:
#pred_orig => predicted class

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
#exp = explainer.explain_instance(normalized_img[0], 
#                                 model.predict, 
#                                 top_labels=5, #might not need
#                                 hide_color=0, 
#                                 num_samples=1000)

In [ ]:
# helper functions

def generate_prediction_sample(exp, exp_class, weight = 0.1, show_positive = True, hide_background = True):
    '''
    Method to display and highlight super-pixels used by the black-box model to make predictions
    '''
    plt.figure()
    image, mask = exp.get_image_and_mask(exp_class, 
                                         positive_only=show_positive, 
                                         num_features=6, 
                                         hide_rest=hide_background,
                                         min_weight=weight
                                        )
    plt.imshow(mark_boundaries(image, mask))
    plt.axis('off')
    plt.show()
    
#Heatmap
#show how important each super-pixel is to get more granular explaianbility.
def explanation_heatmap(exp, exp_class):
    '''
    Using heat-map to highlight the importance of each super-pixel for the model prediction
    '''
    plt.figure()
    dict_heatmap = dict(exp.local_exp[exp_class])
    heatmap = np.vectorize(dict_heatmap.get)(exp.segments) 
    plt.imshow(heatmap, cmap = 'RdBu', vmin  = -heatmap.max(), vmax = heatmap.max())
    plt.colorbar()
    plt.show()

In [ ]:
i = 0
for img_name, img in X.iterrows():
    # output prediction
    title = f"Image {i+1}"
    prediction = f"Confidence that the image is melanoma: {y_pred[i][0]}"
    i+=1
        
    # get image in correct format
    img_to_display = img.to_numpy()

    exp = explainer.explain_instance(img_to_display, 
                                 model.predict, 
                                 top_labels=5, #might not need
                                 hide_color=0, 
                                 num_samples=1000)
    if expect_flattened_input:
        img_to_display = img_to_display.reshape(img_pixel,img_pixel,depth) # unflatten the image
    
    # display image
    plt.figure()
    plt.imshow(img_to_display)
    plt.title(prediction)
    plt.suptitle(title, fontweight ="bold")
    
    plt.imshow(exp.segments)
    plt.axis('off')
    plt.show()
    
    generate_prediction_sample(exp, exp.top_labels[0], show_positive = True, hide_background = True)
    
    generate_prediction_sample(exp, exp.top_labels[0], show_positive = True, hide_background = False)
    #highlight the contour of the superpixel and include the background
    
    generate_prediction_sample(exp, exp.top_labels[0], show_positive = False, hide_background = False)
    #highlight the positive super-pixels and negative superpixels
    
    explanation_heatmap(exp, exp.top_labels[0])